# Introdução às bibliotecas de modelagem em Python


---



### Interface entre o pandas e o código dos modelos

Um fluxo de trabalho comum para desenvolvimento de modelos é
usar o pandas para carga e limpeza de dados antes de passar para
uma biblioteca de modelagem e construir o modelo propriamente
dito. Uma parte importante do processo de desenvolvimento de
modelos é chamada de *engenharia de características* (feature
engineering) em aprendizado de máquina. Ela pode descrever
qualquer transformação ou análise de dados que extraia
informações de um conjunto de dados brutos, as quais sejam úteis
em um contexto de modelagem. As ferramentas de agregação de
dados e o GroupBy que exploramos neste livro são usados com
frequência em um contexto de engenharia de características.

Embora detalhes sobre uma “boa” engenharia de características
estejam fora do escopo deste livro, apresentarei alguns métodos
para fazer com que a passagem da manipulação de dados com o
pandas para a modelagem seja a mais tranquila possível.

O ponto de contato entre o pandas e outras bibliotecas de análise
em geral são os arrays NumPy. Para transformar um DataFrame em
um array NumPy, utilize a propriedade *.values*:

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],
                     'x1': [0.01, -0.01, 0.25, -4.1, 0.],
                     'y': [-1.5, 0., 3.6, 1.3, -2]})

In [4]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [5]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [6]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

Para converter de volta para um DataFrame, conforme você deve
recordar de capítulos anteriores, podemos passar um ndarray
bidimensional com nomes opcionais para as colunas:

In [7]:
df2 = pd.DataFrame(data.values, columns=['one', 'two', 'three'])

In [8]:
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


OBS: *O atributo .values tem como finalidade ser usado quando seus dados são
homogêneos – por exemplo, todos são do tipo numérico. Se você tiver
dados heterogêneos, o resultado será um ndarray de objetos Python:*

In [9]:
df3 = data.copy()

In [10]:
df3['strings'] = ['a', 'b', 'c', 'd', 'e']

In [11]:
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

Para alguns modelos, talvez você queira usar apenas um
subconjunto das colunas. Recomendo utilizar a indexação *loc* com
*values*:

In [12]:
model_col = ['x0', 'x1']

In [13]:
data.loc[:, model_col].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

Algumas bibliotecas têm suporte nativo para o pandas e fazem parte
dessa tarefa para você automaticamente: realizando a conversão
para NumPy a partir de um DataFrame e associando nomes de
parâmetros de modelos às colunas das tabelas ou Series de saída.
Em outros casos, você terá que executar esse “gerenciamento de
metadados” manualmente.

No Capítulo 12, vimos o tipo *Categorical* do pandas e a função
*pandas.get_dummies*. Suponha que tivéssemos uma coluna não
numérica em nosso conjunto de dados de exemplo:


In [14]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                  categories=['a', 'b'])

In [15]:
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


Se quiséssemos substituir a coluna *'category'* por variáveis dummy,
criaríamos essas variáveis, descartaríamos a coluna *'category'* e
então faríamos uma junção (join) do resultado:

In [16]:
dummies = pd.get_dummies(data.category, prefix='category')

In [17]:
data_with_dummies = data.drop('category', axis=1).join(dummies)

In [18]:
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


Há algumas nuances para a adequação de certos modelos
estatísticos com variáveis dummy. Talvez seja mais simples e
menos suscetível a erros usar o Patsy (assunto da próxima seção)
quando você tiver mais que simples colunas numéricas.

### Criando descrições de modelos com o Patsy

O Patsy ( https://patsy.readthedocs.io ) é uma biblioteca Python para
descrever modelos estatísticos (especialmente modelos lineares)
com uma “sintaxe de fórmula” baseada em pequenas strings,
inspirada (mas não exatamente igual) na sintaxe de fórmulas usada
pelas linguagens de programação estatística R e S.

O Patsy tem bom suporte para especificar modelos lineares nostatsmodels, portanto manterei o foco em algumas das principais
funcionalidades a fim de ajudar você a estar pronto para usá-lo. As
fórmulas do Patsy têm uma sintaxe especial de string com o seguinte
aspecto:

```python
y ~ x0 + x1
```

A sintaxe a + b não significa somar a com b , mas sim que são termos
da matriz de design criada para o modelo. A função patsy.dmatrices
aceita uma string com uma fórmula, junto com um conjunto de
dados (que pode ser um DataFrame ou um dicionário de arrays), e
gera matrizes de design para um modelo linear:

In [19]:
data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],
                     'x1': [0.01, -0.01, 0.25, -4.1, 0.],
                     'y': [-1.5, 0., 3.6, 1.3, -2]})

In [20]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [24]:
!pip install --upgrade patsy

Requirement already up-to-date: patsy in /usr/local/lib/python3.7/dist-packages (0.5.1)


In [25]:
import patsy

In [26]:
y, X = patsy.dmatrices('y ~ x0 + x1', data)

In [27]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [28]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

Essas instâncias de DesignMatrix do Patsy são ndarrays NumPy com
metadados adicionais:

In [29]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [30]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

Você deve estar se perguntando de onde veio o termo Intercept. Essa
é uma convenção para modelos lineares, como o de regressão de
OLS (Ordinary Least Squares, Mínimos Quadrados Ordinários).
Podemos suprimir a interceptação adicionando o termo + 0 ao
modelo:

In [31]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

Os objetos do Patsy podem ser passados diretamente para
algoritmos como *numpy.linalg.lstsq*, que executa uma regressão de
mínimos quadrados ordinários:

In [32]:
coef, resid, _, _ = np.linalg.lstsq(X, y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


Os metadados do modelo são mantidos no atributo *design_info*,
portanto você pode associar novamente os nomes das colunas do
modelo aos coeficientes apropriados a fim de obter uma Series, por
exemplo:

In [33]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [34]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)

In [35]:
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

### Transformações de dados em fórmulas do Patsy

Podemos misturar código Python nas fórmulas do Patsy; ao avaliara fórmula, a biblioteca tentará encontrar as funções usadas no
escopo que as incluem:

In [36]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)

In [37]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

Algumas transformações de variáveis comumente utilizadas incluem
padronização (para média 0 e variância 1) e centralização
(subtração da média). O Patsy tem funções embutidas para isso:

In [38]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)

In [39]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

Como parte de um processo de modelagem, você pode fazer a
adequação de um modelo a um conjunto de dados e então avaliar o
modelo em outro conjunto. Esse pode ser uma porção *reservada* dosdados ou novos dados observados depois. Ao aplicar
transformações como centralização e padronização, tome cuidado
ao usar o modelo para fazer previsões baseadas em novos dados.
Essas transformações são chamadas de *stateful* (com estado) porque
você deve usar estatísticas como média ou desvio-padrão do
conjunto de dados original quando fizer transformações em um novo
conjunto de dados.

A função *patsy.build_design_matrices* é capaz de aplicar transformações
em novos dados *fora da amostra* (out-of-sample) usando as
informações salvas do conjunto de dados original *da amostra* (in-
sample):

In [40]:
new_data = pd.DataFrame({'x0': [6, 7, 8, 9],
                         'x1': [3.1, -0.5, 0, 2.3],
                         'y': [1, 2, 3, 4]})

In [41]:
new_X = patsy.build_design_matrices([X.design_info], new_data)

In [42]:
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

Como o símbolo de adição ( + ) no contexto das fórmulas do Patsy
não significa soma, se quisermos somar colunas de um conjunto de
dados pelo nome, devemos encapsulá-las na função especial *I*:

In [43]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)

In [44]:
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

O Patsy tem várias outras transformações embutidas no módulo
*patsy.builtins*. Consulte a documentação online para obter mais
informações.

Os dados categorizados apresentam uma classe especial de
transformações, que explicarei a seguir.

### Dados categorizados e o Patsy

Dados não numéricos podem ser transformados em uma matriz de
design de modelo de várias maneiras diferentes. Uma abordagem
completa sobre esse assunto está fora do escopo deste livro e seria
mais apropriada para um curso de estatística.

Quando usamos termos não numéricos em uma fórmula do Patsy,
eles são, por padrão, convertidos para variáveis dummy. Se houver
uma interceptação, um dos níveis será deixado de fora a fim de
evitar colinearidade:

In [45]:
data = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
                     'key2': [0, 1, 0, 1, 0, 1, 0, 0],
                     'v1': [1, 2, 3, 4, 5, 6, 7, 8],
                     'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]})

In [46]:
data

,key1,key2,v1,v2
0,a,0,1,-1.0
1,a,1,2,0.0
2,b,0,3,2.5
3,b,1,4,-0.5
4,a,0,5,4.0
5,b,1,6,-1.2
6,a,0,7,0.2
7,b,0,8,-1.7


In [47]:
y, X = patsy.dmatrices('v2 ~ key1', data)

In [48]:
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

Se você omitir a interceptação do modelo, colunas para cada valor
de categoria serão incluídas na matriz de design do modelo:

In [49]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)

In [50]:
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

Colunas numéricas podem ser interpretadas como categorizadas
com a função *C*:

In [51]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)

In [52]:
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

Se vários termos categorizados forem usados em um modelo, a
situação talvez se torne mais complicada, pois você poderá incluir
termos de interação no formato *key1:key2*, os quais poderão ser
utilizados, por exemplo, em modelos de ANOVA (Analysis of
Variance, ou Análise de Variância):

In [53]:
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})

In [54]:
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [55]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)

In [56]:
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [57]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)

In [58]:
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

O Patsy oferece outras maneiras de transformar dados
categorizados, incluindo transformações para termos com uma
ordem em particular. Consulte a documentação online para obter
mais informações.

### Introdução ao statsmodels

O statsmodels ( http://www.statsmodels.org ) é uma biblioteca Python para
adequação de vários tipos de modelos estatísticos; ele realiza testes
estatísticos, além de possibilitar a exploração e a visualização de dados. O statsmodels contém métodos estatísticos frequentistas
mais “clássicos”, enquanto os métodos bayesianos e de
aprendizado de máquina são encontrados em outras bibliotecas.

Alguns tipos de modelos que se encontram no statsmodels incluem:

- modelos lineares, modelos lineares generalizados e modelos
lineares robustos;

- modelos lineares de efeitos mistos;

- métodos de ANOVA (Analysis of Variance, ou Análise de
Variância);

- processos para séries temporais e modelos de estado-espaço;

- métodos de momentos generalizados.

Nas próximas páginas, usaremos algumas ferramentas básicas do
statsmodels e veremos como utilizar as interfaces de modelagem
com fórmulas do Patsy e objetos DataFrame do pandas.

### Estimando modelos lineares

Há vários tipos de modelos de regressão linear no statsmodels, dos
mais básicos (por exemplo, mínimos quadrados ordinários) aos mais
complexos (por exemplo, mínimos quadrados ponderados
iterativamente).

Os modelos lineares no statsmodels têm duas interfaces principais
diferentes: baseadas em array e baseadas em fórmula. Elas são
acessadas por meio das seguintes importações de módulos de API:

```python
import statsmodels.api as sm
import statsmodels.formula.api as smf
```

Para mostrar como usá-las, geraremos um modelo linear a partir de
alguns dados aleatórios:


In [59]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [60]:
def dnorm(mean, var, size=1):
  if isinstance(size, int):
    size = size,
  return mean + np.sqrt(var) * np.random.randn(*size)

In [61]:
np.random.seed(12345)

In [62]:
N = 100
X = np.c_[dnorm(0, 0.4, size=N),
          dnorm(0, 0.4, size=N),
          dnorm(0, 0.4, size=N)]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

In [63]:
y = np.dot(X, beta) + eps

In [66]:
y[:5]

array([ 0.59905517, -0.70213475, -0.06083134, -0.50335742, -0.21660308])

Nesse caso, escrevi o modelo “verdadeiro” com parâmetros beta
conhecidos. No exemplo, *dnorm* é uma função auxiliar para gerar
dados normalmente distribuídos, com uma média e uma variância
específicas. Então agora temos:

In [65]:
X[:5]

array([[-0.12946849, -0.99020862,  0.71308166],
       [ 0.30291036, -0.35578165, -0.35946461],
       [-0.32852189, -0.02065862,  0.19565781],
       [-0.35147471, -0.58755511, -0.36517064],
       [ 1.2432688 , -0.30520574, -0.73910908]])

Uma adequação a um modelo linear em geral é feita com um termo
de interceptação, conforme vimos antes com o Patsy. A função
*sm.add_constant* pode adicionar uma coluna de interceptação em uma
matriz existente:

In [67]:
X_model = sm.add_constant(X)

In [68]:
X_model[:5]

array([[ 1.        , -0.12946849, -0.99020862,  0.71308166],
       [ 1.        ,  0.30291036, -0.35578165, -0.35946461],
       [ 1.        , -0.32852189, -0.02065862,  0.19565781],
       [ 1.        , -0.35147471, -0.58755511, -0.36517064],
       [ 1.        ,  1.2432688 , -0.30520574, -0.73910908]])

A classe *sm.OLS* pode fazer a adequação a uma regressão linear de
mínimos quadrados ordinários:

In [69]:
model = sm.OLS(y, X)

In [70]:
results = model.fit()

In [71]:
results.params

array([0.17826108, 0.20574317, 0.50067241])

O método *summary* em *results* é capaz de exibir uma saída de
diagnóstico do detalhamento do modelo:

In [72]:
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.516
Model:                            OLS   Adj. R-squared (uncentered):              0.501
Method:                 Least Squares   F-statistic:                              34.46
Date:                Tue, 25 May 2021   Prob (F-statistic):                    3.00e-15
Time:                        00:34:46   Log-Likelihood:                         -34.305
No. Observations:                 100   AIC:                                      74.61
Df Residuals:                      97   BIC:                                      82.42
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

Os nomes dos parâmetros nesse caso receberam os nomes
genéricos *x1, x2* e assim por diante. Suponha, por outro lado, que
todos os parâmetros do modelo estejam em um DataFrame:

In [73]:
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])

In [74]:
data['y'] = y

In [75]:
data[:5]

,col0,col1,col2,y
0,-0.129468,-0.990209,0.713082,0.599055
1,0.302910,-0.355782,-0.359465,-0.702135
2,-0.328522,-0.020659,0.195658,-0.060831
3,-0.351475,-0.587555,-0.365171,-0.503357
4,1.243269,-0.305206,-0.739109,-0.216603


Agora podemos usar a API de fórmulas do statsmodels e as strings
de fórmula do Patsy:

In [76]:
results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()

In [77]:
results.params

Intercept    0.033559
col0         0.176149
col1         0.207931
col2         0.510471
dtype: float64

In [79]:
results.tvalues

Intercept    0.952188
col0         3.319754
col1         3.662977
col2         8.840053
dtype: float64

Observe como o statsmodels devolveu o resultado na forma de
Series com os nomes das colunas do DataFrame associados.
Também não é necessário usar *add_constant* quando utilizamos
fórmulas e objetos do pandas.

Se novos dados fora da amostra forem especificados, poderemos
calcular os valores previstos, considerando os parâmetros
estimados pelo modelo:


In [80]:
results.predict(data[:5])

0    0.168865
1   -0.170558
2    0.071272
3   -0.336933
4   -0.188197
dtype: float64

Há várias ferramentas adicionais para análise, diagnósticos e
visualização de resultados de modelos lineares no statsmodels, as
quais você poderá explorar. Há também outros tipos de modelos
lineares além dos mínimos quadrados ordinários.

### Introdução ao scikit-learn

O scikit-learn ( http://scikit-learn.org/) é um dos kits de ferramentas
Python de propósito geral para aprendizado de máquina mais
amplamente utilizado e confiável. Ele contém uma variada seleção
de métodos padrões supervisionados e não supervisionados de
aprendizado de máquina, com ferramentas para seleção e avaliação
de modelos, transformação e carga de dados e persistência de
modelos. Esses modelos podem ser usados para classificação,
clustering, previsão e outras tarefas comuns.

Há excelentes recursos online e impressos, como o scikit-learn e o
TensorFlow, para estudar o aprendizado de máquina e aplicarbibliotecas a fim de resolver problemas do mundo real. Nesta seção,
apresentarei uma rápida amostra do estilo de API do scikit-learn.
Atualmente (quando escrevi este livro), o scikit-learn não tem uma
integração profunda com o pandas, embora haja alguns pacotes
add-on de terceiros em desenvolvimento. O pandas, porém, pode
ser muito útil para tratamento de conjuntos de dados antes da
adequação a um modelo.

Como exemplo, usarei um conjunto de dados atualmente clássico
de uma competição do Kaggle ( https://www.kaggle.com/c/titanic ) sobre as
taxas de sobrevivência de passageiros do Titanic , navio que afundou
em 1912. Carregaremos o conjunto de dados de teste e de
treinamento com o pandas:

In [91]:
path_train = '/content/drive/MyDrive/Análise de Dados/CSVs e TXTs/train.csv'
path_test = '/content/drive/MyDrive/Análise de Dados/CSVs e TXTs/test.csv'

In [92]:
train = pd.read_csv(path_train)

In [93]:
test = pd.read_csv(path_test)

In [94]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [95]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [96]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

Nos exemplos de estatística e de aprendizado de máquina como
esse, uma tarefa típica é prever se um passageiro sobreviveria, com
base nas características dos dados. Fazemos a adequação de um
modelo a um conjunto de dados de *treinamento* e, então, uma
avaliação é realizada em um conjunto de dados de testes que não
estão na amostra.

Gostaria de usar *Age* como um dado de previsão, porém ele tem
dados ausentes. Há algumas maneiras de imputar um valor aosdados ausentes, mas utilizarei uma alternativa simples: a mediana
do conjunto de dados de treinamento será usada para preencher os
nulos nas duas tabelas:

In [97]:
impute_value = train['Age'].median()

In [98]:
train['Age'] = train['Age'].fillna(impute_value)

In [99]:
test['Age'] = train['Age'].fillna(impute_value)

Agora, temos que especificar os nossos modelos. Adicionarei uma
coluna *IsFemale* como uma versão codificada da coluna *'Sex'*:

In [102]:
train['IsFemale'] = (train['Sex'] == 'female').astype(int)

In [103]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,IsFemale
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [104]:
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

In [105]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,IsFemale
0,892,3,"Kelly, Mr. James",male,22.0,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,38.0,1,0,363272,7.0000,NaN,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,26.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,35.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,35.0,1,1,3101298,12.2875,NaN,S,1


Então decidiremos sobre algumas variáveis do modelo e criaremos
arrays NumPy:

In [106]:
predictors = ['Pclass', 'IsFemale', 'Age']

In [107]:
X_train = train[predictors].values

In [108]:
X_test = test[predictors].values

In [109]:
y_train = train['Survived'].values

In [110]:
X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [111]:
y_train[:5]

array([0, 1, 1, 1, 0])

Não farei nenhuma argumentação de que esse seja um bom modelo
nem de que essas características tenham sido definidas de forma
apropriada. Usaremos o modelo *LogisticRegression* do scikit-learn e
criaremos uma instância do modelo:

In [112]:
from sklearn.linear_model import LogisticRegression

In [113]:
model = LogisticRegression()

De modo semelhante ao statsmodels, podemos fazer a adequação
desse modelo aos dados de treinamento utilizando o método fit do
modelo:

In [114]:
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Podemos agora fazer previsões para o conjunto de dados de teste
usando *model.predict*:

In [115]:
y_predict = model.predict(X_test)

In [116]:
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0])

Se você tivesse os valores verdadeiros para o conjunto de dados de
teste, seria possível calcular um percentual de exatidão ou outra
métrica de erro:

```python
(y_true == y_predict).mean()
```

Na prática, em geral, há muitas camadas adicionais de
complexidade no treinamento do modelo. Muitos modelos têm
parâmetros que podem ser ajustados, e há técnicas como a validação
cruzada , que podem ser usadas para ajuste de parâmetros a fim de
evitar uma adequação exagerada nos dados de treinamento. Com
frequência isso pode resultar em um desempenho melhor para as
previsões ou em mais robustez para os dados novos.

A validação cruzada funciona por meio da separação dos dados de
treinamento a fim de simular uma previsão fora da amostra. Com
base na pontuação da exatidão do modelo, como o erro quadrático
médio, podemos fazer uma busca de grade (grid search) nos
parâmetros do modelo. Alguns modelos, como a regressão logística,têm classes de estimativa com validação cruzada embutida. Por
exemplo, a classe *LogisticRegressionCV* pode ser usada com um
parâmetro que indica o nível de granularidade que uma busca de
grade deve ter no parâmetro *C* de regularização do modelo:

In [117]:
from sklearn.linear_model import LogisticRegressionCV

In [118]:
model_cv = LogisticRegressionCV(10)

In [119]:
model_cv.fit(X_train, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='auto', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

Para fazer uma validação cruzada manualmente, podemos usar a
função auxiliar *cross_val_score*, que cuida do processo de separação
dos dados. Por exemplo, para fazer a validação cruzada de nosso
modelo com quatro conjuntos separados dos dados de treinamento
sem intersecção, podemos executar o seguinte:

In [120]:
from sklearn.model_selection import cross_val_score

In [121]:
model = LogisticRegression(C=10)

In [122]:
scores = cross_val_score(model, X_train, y_train, cv=4)

In [123]:
scores

array([0.77578475, 0.79820628, 0.77578475, 0.78828829])

A métrica de pontuação padrão é dependente do modelo, mas é
possível escolher uma função explícita de pontuação. Modelos de
validação cruzada demoram mais para efetuar o treinamento; no
entanto, muitas vezes podem levar a um desempenho melhor do
modelo.